Tarea 1

In [ ]:
# --- CELDA DE PRUEBA MANUAL ---
# (Borra esta celda antes de ponerla en el Job final)

# 1. Pega la configuración COMPLETA Y VÁLIDA
test_config_json = """
{
  "name": "df1",
  "sources": [
    {
      "name": "person_inputs",
      "path": "/Volumes/workspace/prueba_tecnica/data/inputs/events/person/*",
      "format": "JSON"
    },
    {
      "name": "employees_inputs",
      "path": "/Volumes/workspace/prueba_tecnica/data/inputs/events/employees/*",
      "format": "JSON"
    }
  ],
  "transformations": [
    {
      "name": "validation",
      "type": "validate_fields",
      "params": { 
        "input": "person_inputs", 
        "validations": [
          {
            "field": "office",
            "validations": ["notEmpty"]
          },
          {
            "field": "age",
            "validations": ["notNull"]
          }
        ] 
      }
    },
    {
      "name": "ok_with_date",
      "type": "add_fields",
      "params": { 
        "input": "validation_ok", 
        "addFields": [
          {
            "name": "dt",
            "function": "current_timestamp"
          }
        ] 
      }
    }
  ],
  "sinks": [
    {
      "input": "ok_with_date",
      "name": "raw_ok",
      "path": "/Volumes/workspace/prueba_tecnica/silver/person",
      "format": "DELTA",
      "saveMode": "OVERWRITE"
    },
    {
      "input": "validation_ko",
      "name": "raw_ko",
      "path": "/Volumes/workspace/prueba_tecnica/discards/person",
      "format": "DELTA",
      "saveMode": "OVERWRITE"
    }
  ]
}
"""

# 2. Esto simula lo que hace el Job:
dbutils.widgets.text("dataflow_config_json", test_config_json)
print("Widget 'dataflow_config_json' rellenado con datos de prueba VÁLIDOS.")

In [ ]:
import sys
import os

# 1. Obtener la raíz del proyecto (en Repos, os.getcwd() es la raíz)
project_root = os.getcwd()

# 2. Añadir la raíz al path de Python
if project_root not in sys.path:
    sys.path.append(project_root)
    
# Databricks Notebook: 01_job_setup_orchestrator
import json
import logging

# IMPORTANTE: Asume que tu código 'src' está disponible
# (ej. porque estás usando Databricks Repos)
from src import utils 

log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Widgets ---
# Creamos un widget para la ruta del config, así es configurable
dbutils.widgets.text("config_path", "/Volumes/workspace/prueba_tecnica/config/metadata.json", "Ruta al metadata.json")
config_path = dbutils.widgets.get("config_path")

log.info(f"Tarea 1: Iniciando. Leyendo metadatos desde {config_path}")

# 1. Cargar la configuración
# (No necesitamos crear Spark, ya existe como 'spark')
config = utils.load_config(config_path)

if config and 'dataflows' in config:
    dataflows_list = config['dataflows']
    log.info(f"Metadatos cargados. {len(dataflows_list)} dataflows encontrados.")
    
    # 2. Pasar la lista de dataflows a la Tarea 2 (ForEach)
    # ¡Esta es la magia! 'dbutils.jobs.taskValues.set' pasa valores
    dbutils.jobs.taskValues.set(key="dataflows_config_list", value=dataflows_list)
    
    print(f"Lista de {len(dataflows_list)} dataflows pasada a la siguiente tarea.")
    
else:
    log.error("No se pudieron cargar los metadatos o la clave 'dataflows' no existe.")
    raise ValueError("Error al cargar la configuración de dataflows.")